<a href="https://colab.research.google.com/github/Pooja-Informatics/symmetrical-orchid/blob/main/all_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All data files

**Mounting drive folders to collab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
filePath ='/content/drive/MyDrive/Data_training/train_data_all_files/clean_trainset_wav'
filePath ='/content/drive/MyDrive/Data_training/train_data_all_files/noisy_trainset_wav'

Mounted at /content/drive


**Training**

In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

noisy_folder = '/content/drive/MyDrive/Data_training/train_data_all_files/noisy_trainset_wav'
clean_folder = '/content/drive/MyDrive/Data_training/train_data_all_files/clean_trainset_wav'
frame_length = 100  # Length of each frame in samples
batch_size = 32

# Function to load audio from file
def load_audio(file_path):
    audio, _ = tf.audio.decode_wav(tf.io.read_file(file_path))
    audio = tf.squeeze(audio, axis=-1)
    return audio.numpy()

# Function to convert audio into frames
def convert_audio_to_frames(audio, frame_length):
    num_frames = len(audio) // frame_length
    frames = np.array_split(audio[:num_frames * frame_length], num_frames)
    return frames

# Function to create the data generator
def create_data_generator(noisy_folder, clean_folder, frame_length, batch_size):
    noisy_files = os.listdir(noisy_folder)
    clean_files = os.listdir(clean_folder)
    assert len(noisy_files) == len(clean_files)

    num_examples = len(noisy_files)
    num_batches = num_examples // batch_size

    while True:
        np.random.shuffle(noisy_files)
        np.random.shuffle(clean_files)

        for i in range(num_batches):
            batch_noisy = []
            batch_clean = []

            for j in range(i * batch_size, (i + 1) * batch_size):
                noisy_path = os.path.join(noisy_folder, noisy_files[j])
                clean_path = os.path.join(clean_folder, clean_files[j])

                noisy_audio = load_audio(noisy_path)
                clean_audio = load_audio(clean_path)

                # Adjust frame splitting to match the number of frames in both noisy and clean audio
                num_frames = min(len(noisy_audio), len(clean_audio)) // frame_length
                noisy_frames = convert_audio_to_frames(noisy_audio, frame_length)[:num_frames]
                clean_frames = convert_audio_to_frames(clean_audio, frame_length)[:num_frames]

                batch_noisy.extend(noisy_frames)
                batch_clean.extend(clean_frames)

            batch_noisy = np.array(batch_noisy)
            batch_clean = np.array(batch_clean)

            yield batch_noisy, batch_clean

# Define the Wave-U-Net architecture
def wave_u_net(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder
    conv1 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(inputs)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(pool1)
    pool2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(pool2)
    pool3 = MaxPooling1D(pool_size=2)(conv3)

    # Decoder
    up4 = UpSampling1D(size=2)(pool3)
    up4_padded = ZeroPadding1D(padding=(1, 0))(up4)  # Padding added here
    concat4 = Concatenate()([conv3, up4_padded])
    conv4 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(concat4)

    up5 = UpSampling1D(size=2)(conv4)
    up5_padded = ZeroPadding1D(padding=(1, 0))(up5)  # Padding added here
    concat5 = Concatenate()([conv2, up5])
    conv5 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(concat5)

    up6 = UpSampling1D(size=2)(conv5)
    up6_padded = ZeroPadding1D(padding=(1, 0))(up6)  # Padding added here
    concat6 = Concatenate()([conv1, up6])
    conv6 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(concat6)

    # Output
    output = Conv1D(filters=1, kernel_size=3, padding='same')(conv6)

    model = Model(inputs=inputs, outputs=output)
    return model

# Define the input shape for the Wave-U-Net model
input_shape = (frame_length, 1)

# Create the Wave-U-Net model
model = wave_u_net(input_shape)
model.compile(optimizer='adam', loss='mse')

# Create the data generator
data_generator = create_data_generator(noisy_folder, clean_folder, frame_length, batch_size)

# Train the model using the data generator
model.fit(data_generator, steps_per_epoch=10, epochs=1)


10/10 [==============================] - 229s 23s/step - loss: 0.0040


**Testing**


Audio folder converted to list of arrays


In [ ]:
import os
import numpy as np
import librosa

def convert_folder_to_arrays(folder_path):
    audio_arrays = []

    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Load the audio file using librosa
        audio, _ = librosa.load(file_path, sr=None)

        # Append the audio array to the list
        audio_arrays.append(audio)

    return audio_arrays

def convert_arrays_to_frames(audio_arrays, frame_length):
    list_frames = []

    # Iterate over the audio arrays
    for audio in audio_arrays:
        num_frames = len(audio) // frame_length

        audio_frames = np.array_split(audio[:num_frames * frame_length], num_frames)
        list_frames.extend(audio_frames)

    return list_frames

# Define the folder path
folder_path = '/content/drive/MyDrive/Data_training/train_data_all_files/noisy_testset_wav'

# Converted the folder of audio files into a list of arrays
audio_arrays = convert_folder_to_arrays(folder_path)

# Defineed the frame length
frame_length =100
#16000

# Converted the list of arrays into a list of frames
list_frames = convert_arrays_to_frames(audio_arrays, frame_length)

To get Predicted frames from model

In [ ]:
import numpy as np

# Converted the list of frames to a NumPy array
frames_array = np.array(list_frames)

# Reshaped the frames array
reshaped_frames = frames_array.reshape((-1, frame_length, 1))

# Predicted using the model
predictions = model.predict(reshaped_frames)


4192/4192 [==============================] - 65s 15ms/step


Function to convert audio folder into list of arrays of audios

In [ ]:
import os
import soundfile as sf

folder_path = '/content/drive/MyDrive/Data_training/train_data_all_files/noisy_testset_wav'
# Replace with the path to your folder

audio_arrays = []  # List to store the audio arrays
audio_filenames = []  # List to store the audio filenames

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Read the audio file and store as array
    audio, _ = sf.read(file_path)

    # Append the audio array to the list
    audio_arrays.append(audio)

    # Append the filename to the list
    audio_filenames.append(filename)

# Print the audio files converted to arrays along with their filenames
for i, audio in enumerate(audio_arrays):
    filename = audio_filenames[i]
    print(f"Audio file {i+1}: {filename}")
    print(audio)
    print()  # Add an empty line between audio arrays





1.   Function to convert list of arrays of audios into list of frames
2.   Function to find total number of frames in each audio array
1.   Function to store number of frames in a array
  








In [ ]:
audio_arrays  # list of audio arrays

frame_length =100

# List to store the frames for each audio array
audio_frames = []

# Array to store the number of frames for each audio array
num_frames_array = []

# Iterate over the audio arrays
for audio in audio_arrays:
    # Calculate the number of frames for each audio array
    num_frames = len(audio) // frame_length
    num_frames_array.append(num_frames)

    # Split the audio array into frames
    frames = [audio[i*frame_length : (i+1)*frame_length] for i in range(num_frames)]

    # Add the frames to the list
    audio_frames.append(frames)

    # Print the total number of frames for each audio array
    print(f"Total number of frames in audio array: {num_frames}")

# we can Print the list of frames for each audio array



Storing predicted frames from model into final respective frames with frame numbers

In [ ]:
import numpy as np

def reshape_frames(frames_list, num_frames_array):
    reshaped_frames = []
    start_idx = 0
    for num_frames in num_frames_array:
        end_idx = start_idx + num_frames
        audio_frames = frames_list[start_idx:end_idx]
        reshaped_frames.append(audio_frames)
        start_idx = end_idx
    return reshaped_frames

# Example usage
frames_list =predictions
num_frames_array = np.array(num_frames_array)
reshaped_frames = reshape_frames(frames_list, num_frames_array)

# Print the reshaped frames
#for i, frames in enumerate(reshaped_frames):
   #print(f"Frames for audio {i+1}:")
    #for frame in frames:
       #print(frame)
    #print("---")


**Storing audio files into drive folder**

In [ ]:
import os
import soundfile as sf
import numpy as np

def convert_frames_to_audio(num_frames_array, audio_frames, output_folder, sampling_rate, file_names):

    for i, frames in enumerate(audio_frames):
        num_frames = num_frames_array[i]
        frame_length = len(frames[0])  # Assuming all frames have the same length

        # Concatenate the frames to reconstruct the audio array
        audio = np.concatenate(frames)

        # Calculate the total length of the audio in samples
        audio_length = num_frames * frame_length

        # Get the corresponding file name
        output_filename = file_names[i]

        # Set the output file path
        output_path = os.path.join(output_folder, output_filename)

        # Save the audio file with the correct sampling rate and subtype
        sf.write(output_path, audio[:audio_length], sampling_rate, 'PCM_16')

        print(f"Saved audio file {output_filename} to {output_folder}")


# Example usage
num_frames_array =num_frames_array   # Example array of number of frames
audio_frames = reshaped_frames  # Example list of frames
output_folder = '/content/drive/MyDrive/Data_training/train_data_all_files/output_wav'  # Output folder path
sampling_rate = 44100
file_names =  audio_filenames# Array of file names

convert_frames_to_audio(num_frames_array, audio_frames, output_folder, sampling_rate, file_names)

**Evaluation**

In [ ]:
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import csv

# Variables
predDataPath =  'output_wav/'
#'Enh_clean/' # Predicted data file directory
cleanDataPath = 'clean_testset_wav/' # Reference/clean data file directory

# Mounting GoogleDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
filePath =  '/content/drive/MyDrive/Data_training/train_data_all_files/'
# SNR criteria
def signal_noise_ratio(signal, noise):
  if len(signal) != len(noise): # For different lengths signals
    if len(signal) > len(noise):
      signal = signal[0:len(noise)]
    else:
      noise = noise[0:len(signal)]
  if len(signal) == len(noise): # For equal length signals
    tmp = signal / len(signal)
    se = np.sum(np.multiply(tmp, tmp))
    tmp = noise / len(noise)
    ne = np.sum(np.multiply(tmp, tmp))
    snr = 10 * np.log10(se/ne)
  else:
    snr = np.nan
  return snr


# Reading directories
def dir_file_reading(drivePath, dataFolder):
  # drivePath - the main working directory
  # dataFolder - data folder
  fullDataPath = drivePath + dataFolder
  fileList = os.listdir(fullDataPath)
  data = []
  files = []
  for file in fileList:
    audio, fs = sf.read(fullDataPath + file) # Nuskaitom duomenis
    data.append(audio)
    files.append(file)
  return data, files

# Reading the directories
predData, predFiles = dir_file_reading(filePath, predDataPath)
cleanData, cleanFiles = dir_file_reading(filePath, cleanDataPath)
# Reference/clean and predicted signal differences are evaluated

print('Saving result file...')
# Creation and saving file
now = datetime.now()
fileName = now.strftime("Results_%Y-%m-%d_%H:%M.csv")
data = ['File', 'SNR']

for i in range(len(predFiles)):
  file = predFiles[i]
  print(file)
  print(cleanFiles)
  print('--------------')
  if file in cleanFiles:
    for j in range(len(cleanFiles)):
      if file == cleanFiles[j]:
        if len(cleanData[j]) > len(predData[i]):
          cleanData[j] = cleanData[j][0:len(predData[i])]
        else:
          predData[i] = predData[i][0:len(cleanData[j])]
        noise = np.subtract(predData[i], cleanData[j])
        snr = signal_noise_ratio(cleanData[j], noise)
        print(snr)
        data = np.vstack([data, [file, snr]])
        break

with open(filePath + fileName, 'w') as csvFile:
  csvWriter = csv.writer(csvFile)
  csvWriter.writerows(data) # Data
